In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("data/installments_payments.csv")

In [3]:
def group_observations_by_month(var):
    
    start_date = var.iloc[0] - 0.1
    end_date = var.iloc[-1] + 0.1
    
    # make bins using the first & last days, with 30 days interval
    my_bin = list(np.arange(start_date, 
                            end_date, 
                            30,
                            ))

    output_var = np.digitize(var,
                             bins = my_bin)
    
    return output_var

In [4]:
# create identifier for a month for all credits
df["month_group"] = df.groupby(["SK_ID_CURR", "SK_ID_PREV"])[["DAYS_INSTALMENT"]].transform(group_observations_by_month)

In [ ]:
df.head(5)

In [17]:
# group activities together by month
df_month_level = df.groupby(["SK_ID_CURR", 
                            "SK_ID_PREV", 
                            "month_group",
                           ]).agg({
    "DAYS_INSTALMENT" : np.mean,
    "DAYS_ENTRY_PAYMENT" : np.mean,
    "AMT_INSTALMENT" : "first",
    "AMT_PAYMENT" : np.sum,
})

In [6]:
df_month_level.head(5)

DAYS_INSTALMENT  DAYS_ENTRY_PAYMENT  \
SK_ID_CURR SK_ID_PREV month_group                                        
100001     1369693    1                    -1709.0             -1715.0   
                      2                    -1679.0             -1715.0   
                      3                    -1634.0             -1644.0   
           1851984    1                    -2916.0             -2916.0   
                      2                    -2871.0             -2865.5   

                                   AMT_INSTALMENT  AMT_PAYMENT  
SK_ID_CURR SK_ID_PREV month_group                               
100001     1369693    1                  3951.000     3951.000  
                      2                  3951.000     3951.000  
                      3                 21348.900    21348.900  
           1851984    1                  3982.050     3982.050  
                      2                  7962.975     7962.975

In [7]:
# group activities together by previous credit
df_prev_level = df.groupby(["SK_ID_CURR", 
                            "SK_ID_PREV",
                           ]).agg({
    "DAYS_INSTALMENT" : np.mean,
    "DAYS_ENTRY_PAYMENT" : np.mean,
    "AMT_INSTALMENT" : np.sum,
    "AMT_PAYMENT" : np.sum,
})

In [8]:
df_prev_level.head(5)

DAYS_INSTALMENT  DAYS_ENTRY_PAYMENT  AMT_INSTALMENT  \
SK_ID_CURR SK_ID_PREV                                                        
100001     1369693             -1664.0        -1679.500000       29250.900   
           1851984             -2886.0        -2882.333333       11945.025   
100002     1038818              -295.0         -315.421053      219625.695   
100003     1810518              -626.0         -630.428571     1150977.330   
           2396755             -2145.0        -2151.750000       80773.380   

                       AMT_PAYMENT  
SK_ID_CURR SK_ID_PREV               
100001     1369693       29250.900  
           1851984       11945.025  
100002     1038818      219625.695  
100003     1810518     1150977.330  
           2396755       80773.380

In [14]:
# aggregate previous credit activities to the mortage
df_curr_level = df.groupby(["SK_ID_CURR"]).agg({
    "DAYS_ENTRY_PAYMENT" : np.mean,
    "DAYS_INSTALMENT" : np.mean,
    "AMT_INSTALMENT" : [np.sum, np.mean],
    "AMT_PAYMENT" : [np.std, np.sum],
})

df_curr_level["MEAN_DAYS_LATE"] = df_curr_level["DAYS_ENTRY_PAYMENT"]["mean"] - df_curr_level["DAYS_INSTALMENT"]["mean"]
df_curr_level["MONEY_OWED"] = df_curr_level["AMT_INSTALMENT"]["sum"] - df_curr_level["AMT_PAYMENT"]["sum"]
df_curr_level["PERC_INSTALMENT_OWED"] = df_curr_level["MONEY_OWED"] / df_curr_level["AMT_INSTALMENT"]["mean"]

In [15]:
df_curr_level.head(5)

DAYS_ENTRY_PAYMENT DAYS_INSTALMENT AMT_INSTALMENT                \
                         mean            mean            sum          mean   
SK_ID_CURR                                                                   
100001           -2195.000000    -2187.714286      41195.925   5885.132143   
100002            -315.421053     -295.000000     219625.695  11559.247105   
100003           -1385.320000    -1378.160000    1618864.650  64754.586000   
100004            -761.666667     -754.000000      21288.465   7096.155000   
100005            -609.555556     -586.000000      56161.845   6240.205000   

              AMT_PAYMENT              MEAN_DAYS_LATE MONEY_OWED  \
                      std          sum                             
SK_ID_CURR                                                         
100001        5076.676624    41195.925      -7.285714        0.0   
100002       10058.037722   219625.695     -20.421053        0.0   
100003      110542.592300  1618864.650      -7.160000        0.0   
100004        3011.871810    21288.465      -7.666667        0.0   
100005        4281.015000    56161.845     -23.555556        0.0   

           PERC_INSTALMENT_OWERD  
                                  
SK_ID_CURR                        
100001                       0.0  
100002                       0.0  
100003                       0.0  
100004                       0.0  
100005                       0.0